In [ ]:
# -*- coding: utf-8 -*-
from hyperparams import Hyperparams as hp
import os
import codecs
from jamo import h2j, j2hcj
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import librosa
import matplotlib.pyplot as plt

In [ ]:
trascript = hp.transcript_pos
lines = codecs.open(trascript, 'r', 'utf-8').readlines()

if not (os.path.isdir(hp.mels_dir)):
    os.mkdir(hp.mels_dir)
    print('{%s} does not exist, created {%s}'.format(hp.mels_dir, hp.mels_dir))
    
if not (os.path.isdir(hp.mags_dir)):
    os.mkdir(hp.mags_dir)
    print('{%s} does not exist, created {%s}'.format(hp.mags_dir, hp.mags_dir))

In [ ]:
def load_vocab():
    char2idx = {char: idx for idx, char in enumerate(hp.vocab)}
    idx2char = {idx: char for idx, char in enumerate(hp.vocab)}
    return char2idx, idx2char

In [ ]:
fnames, texts, secs, text_lengths = [], [], [], []
char2idx, idx2char = load_vocab();

for line in tqdm(lines):
    line = j2hcj(line)
    fname, _, text, sec = line.strip().split('|')
    encodedText = [char2idx[char] for char in text]
    encodedText = np.array(encodedText, np.int32)#.tostring()
    fnames.append(fname); texts.append(encodedText)
    secs.append(float(sec)); text_lengths.append(len(encodedText))
    
fnames = np.asarray(fnames)
texts = np.asarray(texts)
secs = np.asarray(secs)

In [ ]:
#Random Selection
randIdx = np.random.choice(range(len(lines)), 20)
randIdx.sort()
print(randIdx)

fnames = fnames[randIdx]
texts = texts[randIdx]
secs = secs[randIdx]

In [ ]:
maxLen, minLen = max(text_lengths), min(text_lengths)

In [ ]:
def spectrogram(wav, nsc, nov, fs):
    
    
    S = librosa.feature.melspectrogram(y=wav, sr=fs, n_fft=nsc, hop_length=nov, power=2.0)
    dbS = 20 * np.log10(np.maximum(S, hp.eps))
    
    
    return dbS

def mel_spectrogram(wav, nsc, nov, fs):
    
    
    S = librosa.feature.melspectrogram(y=wav, sr=fs, n_fft=nsc, hop_length=nov, power=2.0, n_mels = hp.n_mels)
    dbS = 20 * np.log10(np.maximum(S, hp.eps))
    
    
    return dbS

In [ ]:
for i in tqdm(range(0, len(fnames))):
    fname = fnames[i]
    text = texts[i]
    
    npy_name = fname.split('/')[1].replace('wav', 'npy')
    mel_path = os.path.join(hp.mels_dir, npy_name)
    mag_path = os.path.join(hp.mags_dir, npy_name)
    
#     if os.path.isfile(mel_path) and os.path.isfile(mag_path):
    if False:
        
        mag = np.load(mag_path)
        mel = np.load(mel_path) 
        
    else :
    
        fpath = os.path.join(hp.data_dir, fname)
        wav, fs = librosa.core.load(fpath, mono=True)
        nsc = np.int(fs * hp.nsc_sec)
        nov = np.int(fs * hp.nov_sec)
        mag = spectrogram(wav, nsc, nov, fs)
        mel = mel_spectrogram(wav, nsc, nov, fs)
        mag_coef = np.mean(mag, axis=0)
        
        active = np.where(mag_coef > hp.db_limit)[0]
        
        first = active[0]
        last = active[-1] + 1
        
        if first - hp.offset >= 0:
            first = first - hp.offset
        else:
                first = 0
            
        if last + hp.offset < len(mag_coef):
            last = last + hp.offset
        else:
            last = len(mag_coef)
            
        mag = mag[:, first:last]
        mel = mel[:, first:last]
        
        mag = mag / hp.max_db
        mel = mel / hp.max_db
        
        print('{:d}:{:d}'.format(first, last))
        
        np.save(mag_path, mag)
        np.save(mel_path, mel)
    
    print(fname)
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(mag, origin='lower')
    plt.colorbar()
    plt.subplot(1, 3, 2)
    plt.imshow(mel, origin='lower')
    plt.colorbar()
    plt.subplot(1, 3, 3)
    plt.plot(mag_coef)
    plt.show()

In [ ]:
mag.shape

In [ ]:
with tf.variable_scope("embedding", reuse=tf.AUTO_REUSE):
    lookup_table = tf.get_variable('lookup_table', 
                                   dtype=tf.float32, 
                                   shape=[len(hp.vocab), hp.embed_size],
                                   initializer=tf.truncated_normal_initializer(mean=0.0, stddev=0.01))

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
sess.run(lookup_table)

In [ ]:
embed_texts = []

for i in range(len(texts)):
    text = texts[i]
    embed_text = tf.nn.embedding_lookup(lookup_table, text)
    embed_text = tf.expand_dims(embed_text, 0)
    embed_texts.append(embed_text)

In [ ]:
embed_text

In [ ]:
lookup_table.name

In [ ]:
encodedText

In [ ]:
x = sess.run(embed_texts)

In [ ]:
plt.imshow(x[5][0, :, :])

In [ ]:
prenet_results = []

for i in tqdm(range(len(embed_texts))):
    embed_text = embed_texts[i]
    with tf.variable_scope("prenet", reuse=tf.AUTO_REUSE):
        outputs = tf.layers.dense(embed_text, units=hp.num_prenet_node_1, activation=tf.nn.relu, name="dense1")
        outputs = tf.layers.dropout(outputs, rate=hp.dropout_rate, name="dropout1")
        outputs = tf.layers.dense(outputs, units=hp.num_prenet_node_2, activation=tf.nn.relu, name="dense2")
        outputs = tf.layers.dropout(outputs, rate=hp.dropout_rate, name="dropout2") 
        
    prenet_results.append(outputs)
        

In [ ]:
# sess.run(tf.global_variables_initializer())
# x = sess.run(prenet_results)

In [ ]:
# for i in tqdm(range(len(embed_texts))):
conv_bank_results = []

for i in tqdm(range(len(prenet_results))):
    with tf.variable_scope("conv1d_banks", reuse=tf.AUTO_REUSE):
        prenet_result = prenet_results[i]

        for k in range(1, hp.K + 1):
            with tf.variable_scope("filter_num_{}".format(k)):
                params = {"inputs":prenet_result, "filters":hp.num_k_filter, "kernel_size":k,
                        "dilation_rate":1, "padding":"SAME", "activation":None, 
                        "use_bias":False, "reuse":tf.AUTO_REUSE}
                
                # Works when resue = True
                # For i loop, filter is reused.

                conv_outputs = tf.layers.conv1d(**params)
                if k == 1:
                    conv_bank_outputs = conv_outputs
                else:
                    conv_bank_outputs = tf.concat((conv_bank_outputs, conv_outputs), axis=2)
                
    conv_bank_results.append(conv_bank_outputs)
                
#     print(prenet_outputs)


In [ ]:
sess.run(tf.global_variables_initializer())
sess.run(conv_bank_results)